In [5]:
import cv2
import numpy as np
from IPython.display import display, HTML
import base64

video_path = r'E:\DeepLearningTMP\LinchCV\RM(DPLandCV)\123456789.mp4'
video = cv2.VideoCapture(video_path)

def show_video(video_path):
    video_html = """
    <video width="800" height="600" controls>
        <source src="data:video/mp4;base64,{}" type="video/mp4">
    </video>
    """.format(base64.b64encode(open(video_path, "rb").read()).decode("utf-8"))
    display(HTML(video_html))

# Set up video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path = 'output_video.mp4'
output_video = cv2.VideoWriter(output_video_path, fourcc, 20.0, (640, 480))

current_frame = 0

while True:
    ret, frame = video.read()
    if not ret:
        break
    
    blue, _, _ = cv2.split(frame)
    ret2, binary = cv2.threshold(blue, 220, 255, 0)
    Gaussian = cv2.GaussianBlur(binary, (5, 5), 0)
    
    draw_img = Gaussian.copy()
    whole_h, _ = binary.shape[:2]
    
    _,contours, _ = cv2.findContours(image=draw_img, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, key=lambda c: cv2.contourArea(c), reverse=True)[:5]
    
    width_array = []
    height_array = []
    point_array = []
    
    for cont in contours:
        x, y, w, h = cv2.boundingRect(cont)
        try:
            if h / w >= 2 and h / whole_h > 0.1 and h > w:
                width_array.append(w)
                height_array.append(h)
                point_array.append([x, y])
        except ZeroDivisionError:
            continue
    
    point_near = [0, 0]
    min_value = 10000
    
    for i in range(len(width_array) - 1):
        for j in range(i + 1, len(width_array)):
            value = abs(width_array[i] * height_array[i] - width_array[j] * height_array[j])
            if value < min_value:
                min_value = value
                point_near[0] = i
                point_near[1] = j
    
    try:
        rectangle1 = point_array[point_near[0]]
        rectangle2 = point_array[point_near[1]]
        point1 = [rectangle1[0] + width_array[point_near[0]] / 2, rectangle1[1]]
        point2 = [rectangle1[0] + width_array[point_near[0]] / 2, rectangle1[1] + height_array[point_near[0]]]
        point3 = [rectangle2[0] + width_array[point_near[1]] / 2, rectangle2[1]]
        point4 = [rectangle2[0] + width_array[point_near[1]] / 2, rectangle2[1] + height_array[point_near[1]]]
        
        x = np.array([point1, point2, point4, point3], np.int32)
        box = x.reshape((-1, 1, 2)).astype(np.int32)
        cv2.polylines(frame, [box], True, (0, 255, 0), 2)
        output_video.write(frame)
    except IndexError:
        continue

# Release resources
video.release()
output_video.release()
cv2.destroyAllWindows()

# Display a link to download the output video
output_video_link = f'<a href="{output_video_path}" download>Click here to download the output video</a>'
display(HTML(output_video_link))
